# Data Cleaning with Pandas

## 1.&nbsp; Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', None) # to increase the width of the columns
pd.set_option('display.max_rows', 60000)

### 1.1.&nbsp; Import the csv file to DataFrame

In [3]:
# intermittent-renewables-production-france.csv
url = "https://drive.google.com/file/d/1bs3Dve-Z_OzKC2ownJ7HD-QcFUmmt3C3/view?usp=sharing"
DATA_PATH = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
france = pd.read_csv(DATA_PATH)

In [4]:
france_df = france.copy()

## 2.&nbsp; DataFrames exploration

In [5]:
france_df.shape

(59806, 9)

In [6]:
france_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59806 entries, 0 to 59805
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date and Hour  59806 non-null  object 
 1   Date           59806 non-null  object 
 2   StartHour      59806 non-null  object 
 3   EndHour        59806 non-null  object 
 4   Source         59806 non-null  object 
 5   Production     59804 non-null  float64
 6   dayOfYear      59806 non-null  int64  
 7   dayName        59806 non-null  object 
 8   monthName      59806 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 4.1+ MB


In [7]:
france_df.head()

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
0,2020-07-22 20:00:00+02:00,2020-07-22,20:00:00,21:00:00,Solar,244.0,204,Wednesday,July
1,2020-07-23 07:00:00+02:00,2020-07-23,07:00:00,08:00:00,Solar,223.0,205,Thursday,July
2,2020-07-23 16:00:00+02:00,2020-07-23,16:00:00,17:00:00,Solar,2517.0,205,Thursday,July
3,2020-07-23 19:00:00+02:00,2020-07-23,19:00:00,20:00:00,Solar,658.0,205,Thursday,July
4,2020-07-23 23:00:00+02:00,2020-07-23,23:00:00,24:00:00,Solar,0.0,205,Thursday,July


## 3.&nbsp; Missing values:
* `Production` has 2 missing values

In [8]:
france_df.loc[france_df['Production'].isna(), :]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
36164,2022-03-27 03:00:00+02:00,2022-03-27,02:00:00,03:00:00,Solar,NaN,86,Sunday,March
39321,2022-03-27 03:00:00+02:00,2022-03-27,02:00:00,03:00:00,Wind,NaN,86,Sunday,March


In [9]:
#Delete missing value
#france_df_no_missing = france_df.loc[~france_df.Production.isna(), :]


We will decide on these 2 missing values later.

## 4.&nbsp; Duplicates

In [10]:
france_df.duplicated().sum()

0

We have no duplicate rows in `france_df` DataFrame.

## 5.&nbsp; Data types

In [11]:
# Convert "Date and Hour" to datetime (handling timezone-aware format)
france_df["Date and Hour"] = pd.to_datetime(france_df["Date and Hour"], utc=True)

# Convert "Date" to datetime (date only)
france_df["Date"] = pd.to_datetime(france_df["Date"])

# Ensure "StartHour" is treated as a string before replacing "24:00:00"
france_df["StartHour"] = france_df["StartHour"].astype(str)
# Convert "StartHour" to datetime, allowing "24:00:00"
france_df["StartHour"] = france_df["StartHour"].str.replace("24:00:00", "23:59:99")  # Replace invalid values
# Convert "StartHour" to time format
france_df["StartHour"] = pd.to_datetime(france_df["StartHour"], format="%H:%M:%S")

# Ensure "EndHour" is treated as a string before replacing "24:00:00"
#france_df["EndHour"] = france_df["EndHour"].astype(str)
# Convert "EndHour" to datetime, allowing "24:00:00"
#france_df["EndHour"] = france_df["EndHour"].str.replace("24:00:00", "23:59:99")  # Replace invalid values
# Convert "EndHour" to time format
#france_df["EndHour"] = pd.to_datetime(france_df["EndHour"], format="%H:%M:%S").dt.time

# Ensure "dayName" is properly formatted as categorical (optional)
france_df["dayName"] = pd.Categorical(france_df["dayName"], categories=[
    "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
], ordered=True)

# Ensure "monthName" is properly formatted as categorical (optional)
france_df["monthName"] = pd.Categorical(france_df["monthName"], categories=[
    "January", "February", "March", "April", "May", "June", 
    "July", "August", "September", "October", "November", "December"
], ordered=True)

In [12]:
# Ensure "StartHour" is a string
france_df["EndHour"] = france_df["EndHour"].astype(str)

# Replace "24:00:00" with "00:00:00"
france_df["EndHour"] = france_df["EndHour"].str.replace("24:00:00", "00:00:00")

# Standardize time format: Ensure "H:MM" → "HH:MM:SS"
def fix_time_format(time_str):
    try:
        # If time is missing seconds (e.g., "9:00" → "09:00:00")
        if ":" in time_str and len(time_str.split(":")) == 2:
            time_str += ":00"
        
        # If time is single digit (e.g., "9" → "09:00:00")
        if time_str.isdigit():
            time_str = f"{int(time_str):02d}:00:00"
        
        # Convert to datetime and extract time
        return pd.to_datetime(time_str, format="%H:%M:%S")
    except:
        return None  # Handle unexpected values gracefully

# Apply function to fix incorrect formats
france_df["EndHour"] = france_df["EndHour"].apply(fix_time_format)

In [13]:
france_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59806 entries, 0 to 59805
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   Date and Hour  59806 non-null  datetime64[ns, UTC]
 1   Date           59806 non-null  datetime64[ns]     
 2   StartHour      59806 non-null  datetime64[ns]     
 3   EndHour        59806 non-null  datetime64[ns]     
 4   Source         59806 non-null  object             
 5   Production     59804 non-null  float64            
 6   dayOfYear      59806 non-null  int64              
 7   dayName        59806 non-null  category           
 8   monthName      59806 non-null  category           
dtypes: category(2), datetime64[ns, UTC](1), datetime64[ns](3), float64(1), int64(1), object(1)
memory usage: 3.3+ MB


## 6.&nbsp; Sorting by ``Date and Hour``

In [14]:
# Sort the rows based on "Date and Hour" column in ascending order
france_df = france_df.sort_values(by="Date and Hour", ascending=True)

# Reset the index if needed (optional)
france_df = france_df.reset_index(drop=True)

In [15]:
france_df.head(3)

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
0,2019-12-31 23:00:00+00:00,2020-01-01,1900-01-01 00:00:00,1900-01-01 01:00:00,Solar,0.0,1,Wednesday,January
1,2019-12-31 23:00:00+00:00,2020-01-01,1900-01-01 00:00:00,1900-01-01 01:00:00,Wind,3549.0,1,Wednesday,January
2,2020-01-01 00:00:00+00:00,2020-01-01,1900-01-01 01:00:00,1900-01-01 02:00:00,Solar,0.0,1,Wednesday,January


In [16]:
france_df.tail(3)

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
59803,2023-06-30 20:00:00+00:00,2023-06-30,1900-01-01 22:00:00,1900-01-01 23:00:00,Wind,5140.0,181,Friday,June
59804,2023-06-30 21:00:00+00:00,2023-06-30,1900-01-01 23:00:00,1900-01-01 00:00:00,Wind,6135.0,181,Friday,June
59805,2023-06-30 21:00:00+00:00,2023-06-30,1900-01-01 23:00:00,1900-01-01 00:00:00,Solar,1.0,181,Friday,June


## 7.&nbsp; Separate the rows by ``Source``: ``Solar`` & ``Wind``

In [17]:
# Separate the rows where "Source" is "Solar" and "Wind"
solar_df = france_df[france_df["Source"] == "Solar"]
wind_df = france_df[france_df["Source"] == "Wind"]

In [18]:
solar_df.sample(2)

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
52012,2023-01-19 13:00:00+00:00,2023-01-19,1900-01-01 14:00:00,1900-01-01 15:00:00,Solar,1694.0,19,Thursday,January
56570,2023-04-24 12:00:00+00:00,2023-04-24,1900-01-01 14:00:00,1900-01-01 15:00:00,Solar,4424.0,114,Monday,April


In [19]:
wind_df.sample(2)

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
39777,2022-05-09 14:00:00+00:00,2022-05-09,1900-01-01 16:00:00,1900-01-01 17:00:00,Wind,519.0,129,Monday,May
39363,2022-04-30 23:00:00+00:00,2022-05-01,1900-01-01 01:00:00,1900-01-01 02:00:00,Wind,3601.0,121,Sunday,May


In [20]:
solar_df.head()

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
0,2019-12-31 23:00:00+00:00,2020-01-01,1900-01-01 00:00:00,1900-01-01 01:00:00,Solar,0.0,1,Wednesday,January
2,2020-01-01 00:00:00+00:00,2020-01-01,1900-01-01 01:00:00,1900-01-01 02:00:00,Solar,0.0,1,Wednesday,January
4,2020-01-01 01:00:00+00:00,2020-01-01,1900-01-01 02:00:00,1900-01-01 03:00:00,Solar,0.0,1,Wednesday,January
7,2020-01-01 02:00:00+00:00,2020-01-01,1900-01-01 03:00:00,1900-01-01 04:00:00,Solar,0.0,1,Wednesday,January
8,2020-01-01 03:00:00+00:00,2020-01-01,1900-01-01 04:00:00,1900-01-01 05:00:00,Solar,0.0,1,Wednesday,January


## 8.&nbsp; Missing values imputation

### 8.1. &nbsp; solar_df

In [21]:
solar_df.loc[solar_df['Production'].isna(), :]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37686,2022-03-27 01:00:00+00:00,2022-03-27,1900-01-01 02:00:00,1900-01-01 03:00:00,Solar,NaN,86,Sunday,March


In [22]:
solar_df.loc[solar_df["Date and Hour"] == "2022-03-26 01:00:00+00:00"]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37637,2022-03-26 01:00:00+00:00,2022-03-26,1900-01-01 02:00:00,1900-01-01 03:00:00,Solar,4.0,85,Saturday,March


In [23]:
solar_df.loc[solar_df["Date and Hour"] == "2022-03-28 01:00:00+00:00"]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37735,2022-03-28 01:00:00+00:00,2022-03-28,1900-01-01 03:00:00,1900-01-01 04:00:00,Solar,4.0,87,Monday,March


In [24]:
solar_df.loc[solar_df["Date"] == "2022-03-27"]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37680,2022-03-26 23:00:00+00:00,2022-03-27,1900-01-01 00:00:00,1900-01-01 01:00:00,Solar,4.0,86,Sunday,March
37683,2022-03-27 00:00:00+00:00,2022-03-27,1900-01-01 01:00:00,1900-01-01 02:00:00,Solar,4.0,86,Sunday,March
37685,2022-03-27 01:00:00+00:00,2022-03-27,1900-01-01 03:00:00,1900-01-01 04:00:00,Solar,4.0,86,Sunday,March
37686,2022-03-27 01:00:00+00:00,2022-03-27,1900-01-01 02:00:00,1900-01-01 03:00:00,Solar,NaN,86,Sunday,March
37689,2022-03-27 02:00:00+00:00,2022-03-27,1900-01-01 04:00:00,1900-01-01 05:00:00,Solar,4.0,86,Sunday,March
37690,2022-03-27 03:00:00+00:00,2022-03-27,1900-01-01 05:00:00,1900-01-01 06:00:00,Solar,4.0,86,Sunday,March
37693,2022-03-27 04:00:00+00:00,2022-03-27,1900-01-01 06:00:00,1900-01-01 07:00:00,Solar,4.0,86,Sunday,March
37695,2022-03-27 05:00:00+00:00,2022-03-27,1900-01-01 07:00:00,1900-01-01 08:00:00,Solar,16.0,86,Sunday,March
37696,2022-03-27 06:00:00+00:00,2022-03-27,1900-01-01 08:00:00,1900-01-01 09:00:00,Solar,382.0,86,Sunday,March
37698,2022-03-27 07:00:00+00:00,2022-03-27,1900-01-01 09:00:00,1900-01-01 10:00:00,Solar,1494.0,86,Sunday,March


In [25]:
solar_df.loc[37686, "Production"] = np.float64(4.0)

In [26]:
solar_df.loc[solar_df['Production'].isna(), :]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName


### 8.2. &nbsp; wind_df

In [27]:
wind_df.loc[wind_df['Production'].isna(), :]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37687,2022-03-27 01:00:00+00:00,2022-03-27,1900-01-01 02:00:00,1900-01-01 03:00:00,Wind,NaN,86,Sunday,March


In [28]:
wind_df.loc[wind_df["Date and Hour"] == "2022-03-26 01:00:00+00:00"]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37636,2022-03-26 01:00:00+00:00,2022-03-26,1900-01-01 02:00:00,1900-01-01 03:00:00,Wind,7675.0,85,Saturday,March


In [29]:
wind_df.loc[wind_df["Date and Hour"] == "2022-03-28 00:00:00+00:00"]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37733,2022-03-28 00:00:00+00:00,2022-03-28,1900-01-01 02:00:00,1900-01-01 03:00:00,Wind,1866.0,87,Monday,March


In [30]:
wind_df.loc[wind_df["Date"] == "2022-03-27"]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
37681,2022-03-26 23:00:00+00:00,2022-03-27,1900-01-01 00:00:00,1900-01-01 01:00:00,Wind,8047.0,86,Sunday,March
37682,2022-03-27 00:00:00+00:00,2022-03-27,1900-01-01 01:00:00,1900-01-01 02:00:00,Wind,7374.0,86,Sunday,March
37684,2022-03-27 01:00:00+00:00,2022-03-27,1900-01-01 03:00:00,1900-01-01 04:00:00,Wind,6501.0,86,Sunday,March
37687,2022-03-27 01:00:00+00:00,2022-03-27,1900-01-01 02:00:00,1900-01-01 03:00:00,Wind,NaN,86,Sunday,March
37688,2022-03-27 02:00:00+00:00,2022-03-27,1900-01-01 04:00:00,1900-01-01 05:00:00,Wind,6021.0,86,Sunday,March
37691,2022-03-27 03:00:00+00:00,2022-03-27,1900-01-01 05:00:00,1900-01-01 06:00:00,Wind,5787.0,86,Sunday,March
37692,2022-03-27 04:00:00+00:00,2022-03-27,1900-01-01 06:00:00,1900-01-01 07:00:00,Wind,5317.0,86,Sunday,March
37694,2022-03-27 05:00:00+00:00,2022-03-27,1900-01-01 07:00:00,1900-01-01 08:00:00,Wind,4921.0,86,Sunday,March
37697,2022-03-27 06:00:00+00:00,2022-03-27,1900-01-01 08:00:00,1900-01-01 09:00:00,Wind,4586.0,86,Sunday,March
37699,2022-03-27 07:00:00+00:00,2022-03-27,1900-01-01 09:00:00,1900-01-01 10:00:00,Wind,3383.0,86,Sunday,March


In [31]:
# Get the positional index of the row with index label 37687
pos = wind_df.index.get_loc(37687)

# Ensure that the previous and next positions exist
if pos > 0 and pos < len(wind_df) - 1:
    # Retrieve the Production values from the previous and next rows using .iloc
    val_before = wind_df.iloc[pos - 1]['Production']
    val_after  = wind_df.iloc[pos + 1]['Production']
    
    # Check that both neighboring values are not NaN
    if pd.notna(val_before) and pd.notna(val_after):
        average_value = (val_before + val_after) / 2
        wind_df.loc[37687, 'Production'] = average_value
        print(f"Imputed value {average_value} at index 37687.")
    else:
        print("One or both neighboring values are NaN. Cannot compute the average reliably.")
else:
    print("Index 37687 does not have both a previous and a next row.")

Imputed value 6261.0 at index 37687.


In [32]:
wind_df.loc[wind_df['Production'].isna(), :]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName


## 9.&nbsp; Removing data before 01.04.2020

We exclude data before April 1, 2020, as data for March 2020 is unavailable. This approach minimizes data loss to just three months while retaining three years of data, which is sufficient for our analysis.

In [33]:
# Define the start and end dates
start_date = pd.Timestamp("2020-01-01")
end_date = pd.Timestamp("2020-03-31")

# Create a mask that is True for rows outside the date range
mask = (solar_df['Date'] < start_date) | (solar_df['Date'] > end_date)

# Apply the mask to filter the DataFrame
solar_df = solar_df.loc[mask]

# Optionally, reset the index if needed
solar_df = solar_df.reset_index(drop=True)

In [34]:
solar_df.head(3)

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
0,2020-03-31 22:00:00+00:00,2020-04-01,1900-01-01 00:00:00,1900-01-01 01:00:00,Solar,0.0,92,Wednesday,April
1,2020-03-31 23:00:00+00:00,2020-04-01,1900-01-01 01:00:00,1900-01-01 02:00:00,Solar,0.0,92,Wednesday,April
2,2020-04-01 00:00:00+00:00,2020-04-01,1900-01-01 02:00:00,1900-01-01 03:00:00,Solar,0.0,92,Wednesday,April


In [35]:
# Define the start and end dates
start_date = pd.Timestamp("2020-01-01")
end_date = pd.Timestamp("2020-03-31")

# Create a mask that is True for rows outside the date range
mask_wind = (wind_df['Date'] < start_date) | (wind_df['Date'] > end_date)

# Apply the mask to filter the DataFrame
wind_df = wind_df.loc[mask_wind]

# Optionally, reset the index if needed
wind_df = wind_df.reset_index(drop=True)

In [36]:
wind_df.head(3)

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName
0,2020-03-31 22:00:00+00:00,2020-04-01,1900-01-01 00:00:00,1900-01-01 01:00:00,Wind,6759.0,92,Wednesday,April
1,2020-03-31 23:00:00+00:00,2020-04-01,1900-01-01 01:00:00,1900-01-01 02:00:00,Wind,6293.0,92,Wednesday,April
2,2020-04-01 00:00:00+00:00,2020-04-01,1900-01-01 02:00:00,1900-01-01 03:00:00,Wind,5916.0,92,Wednesday,April


## 10.&nbsp; Save the cleaned DataFrame

In [ ]:
#files.download("orders_cl.csv")
solar_df.to_csv("./data/solar_renewables_production_cl.csv", index=False)
wind_df.to_csv("./data/wind_renewables_production_cl.csv", index=False)